<a href="https://colab.research.google.com/github/lganalon/Website-Portfolio/blob/main/Search_Profiling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from bisect import bisect_left
import copy

# Generating a large matrix of random numbers
np.random.seed(0)  # For reproducibility
matrix = np.random.randint(0, 10000, (1000, 1000))
target = np.random.randint(0, 10000)  # Random target to search for

# Algorithm 1: Sort the matrix, Jump Search followed by Binary Search
def algorithm_1(matrix, target):
    # Work on a copy of the matrix
    matrix_copy = np.sort(matrix.copy(), axis=1)
    # Jump search through the first elements of each row
    n = len(matrix_copy)
    step = int(np.sqrt(n))
    prev = 0
    while matrix_copy[min(step, n) - 1][0] < target:
        prev = step
        step += int(np.sqrt(n))
        if prev >= n:
            return -1
    # Binary search in the identified block
    for row in matrix_copy[prev:min(step, n)]:
        index = binary_search(row, target)
        if index != -1:
            return index
    return -1

def binary_search(arr, target):
    idx = bisect_left(arr, target)
    if idx != len(arr) and arr[idx] == target:
        return idx
    return -1

# Algorithm 2: Sort the matrix, Ternary Search
def ternary_search(arr, left, right, target):
    if right >= left:
        mid1 = left + (right - left) // 3
        mid2 = right - (right - left) // 3
        if arr[mid1] == target:
            return mid1
        if arr[mid2] == target:
            return mid2
        if target < arr[mid1]:
            return ternary_search(arr, left, mid1 - 1, target)
        elif target > arr[mid2]:
            return ternary_search(arr, mid2 + 1, right, target)
        else:
            return ternary_search(arr, mid1 + 1, mid2 - 1, target)
    return -1

def algorithm_2(matrix, target):
    # Work on a copy of the matrix
    matrix_copy = np.sort(matrix.copy(), axis=1)
    for row in matrix_copy:
        index = ternary_search(row, 0, len(row) - 1, target)
        if index != -1:
            return index
    return -1

# Algorithm 3: Sort the matrix, Interpolation Search
def interpolation_search(arr, left, right, target):
    while left <= right and target >= arr[left] and target <= arr[right]:
        if left == right:
            if arr[left] == target:
                return left
            return -1
        pos = left + ((right - left) // (arr[right] - arr[left]) * (target - arr[left]))
        if arr[pos] == target:
            return pos
        if arr[pos] < target:
            left = pos + 1
        else:
            right = pos - 1
    return -1

def algorithm_3(matrix, target):
    # Work on a copy of the matrix
    matrix_copy = np.sort(matrix.copy(), axis=1)
    for row in matrix_copy:
        index = interpolation_search(row, 0, len(row) - 1, target)
        if index != -1:
            return index
    return -1



In [2]:
import timeit

# Timeit for Algorithm 1
time_algorithm_1 = timeit.timeit(lambda: algorithm_1(matrix, target), number=10)
# Timeit for Algorithm 2
time_algorithm_2 = timeit.timeit(lambda: algorithm_2(matrix, target), number=10)
# Timeit for Algorithm 3
time_algorithm_3 = timeit.timeit(lambda: algorithm_3(matrix, target), number=10)

# Print the results
print(f"Algorithm 1 (Jump + Binary Search) Time: {time_algorithm_1}")
print(f"Algorithm 2 (Ternary Search) Time: {time_algorithm_2}")
print(f"Algorithm 3 (Interpolation Search) Time: {time_algorithm_3}")



Algorithm 1 (Jump + Binary Search) Time: 0.8838341360000044
Algorithm 2 (Ternary Search) Time: 1.4910890709999904
Algorithm 3 (Interpolation Search) Time: 1.5751884290000078


In [12]:
#!pip install psutil memory-profiler

In [13]:
from memory_profiler import memory_usage

# Manual memory profiling for Algorithm 1
def run_algorithm_1():
    return algorithm_1(matrix, target)

mem_usage_1 = memory_usage((run_algorithm_1,))  # Memory usage in MiB
print(f"Memory usage for Algorithm 1: {max(mem_usage_1)} MiB")

# Manual memory profiling for Algorithm 2
def run_algorithm_2():
    return algorithm_2(matrix, target)

mem_usage_2 = memory_usage((run_algorithm_2,))
print(f"Memory usage for Algorithm 2: {max(mem_usage_2)} MiB")

# Manual memory profiling for Algorithm 3
def run_algorithm_3():
    return algorithm_3(matrix, target)

mem_usage_3 = memory_usage((run_algorithm_3,))
print(f"Memory usage for Algorithm 3: {max(mem_usage_3)} MiB")




Memory usage for Algorithm 1: 121.71484375 MiB
Memory usage for Algorithm 2: 121.68359375 MiB
Memory usage for Algorithm 3: 121.8984375 MiB


In [9]:
#pip install pyinstrument

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.2/127.2 kB 7.9 MB/s eta 0:00:00


In [10]:
from pyinstrument import Profiler

# PyInstrument for Algorithm 1
profiler = Profiler()
profiler.start()
algorithm_1(matrix, target)
profiler.stop()
print("Algorithm 1 Profile:\n", profiler.output_text())

# PyInstrument for Algorithm 2
profiler.start()
algorithm_2(matrix, target)
profiler.stop()
print("Algorithm 2 Profile:\n", profiler.output_text())

# PyInstrument for Algorithm 3
profiler.start()
algorithm_3(matrix, target)
profiler.stop()
print("Algorithm 3 Profile:\n", profiler.output_text())



Algorithm 1 Profile:
 
  _     ._   __/__   _ _  _  _ _/_   Recorded: 05:32:24  Samples:  5
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.062     CPU time: 0.061
/   _/                      v4.7.3

Profile at <ipython-input-10-7d811d902d25>:5

0.061 Shell.run_code  IPython/core/interactiveshell.py:3512
`- 0.061 <cell line: 6>  <ipython-input-10-7d811d902d25>:1
   `- 0.061 algorithm_1  <ipython-input-1-0f9ffb8fc3c5>:11
      |- 0.056 sort  numpy/core/fromnumeric.py:865
      |     [1 frames hidden]  <built-in>
      |        0.053 ndarray.sort  <built-in>
      `- 0.005 ndarray.copy  <built-in>


Algorithm 2 Profile:
 
  _     ._   __/__   _ _  _  _ _/_   Recorded: 05:32:24  Samples:  10
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.120     CPU time: 0.121
/   _/                      v4.7.3

Profile at <ipython-input-10-7d811d902d25>:5

0.119 Shell.run_ast_nodes  IPython/core/interactiveshell.py:3360
|- 0.061 <cell line: 6>  <ipython-input-10-7d811d902d25>:1
|  `- 0.061 algorithm_

In [11]:
import cProfile

# cProfile for Algorithm 1
cProfile.run('algorithm_1(matrix, target)')

# cProfile for Algorithm 2
cProfile.run('algorithm_2(matrix, target)')

# cProfile for Algorithm 3
cProfile.run('algorithm_3(matrix, target)')


         44 function calls in 0.065 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.065    0.065 <ipython-input-1-0f9ffb8fc3c5>:11(algorithm_1)
        1    0.000    0.000    0.065    0.065 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:861(_sort_dispatcher)
        1    0.000    0.000    0.059    0.059 fromnumeric.py:865(sort)
        1    0.000    0.000    0.065    0.065 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}
       33    0.000    0.000    0.000    0.000 {built-in method builtins.min}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.asanyarray}
        2    0.013    0.006    0.013    0.006 {method 'copy' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.052    0.052    0.052    0.